# <font color="#CA3532">Introducción a TensorFlow (parte II)</font> 

Clasificación multiclase. Ejemplo detallado con el problema Iris.

In [ ]:
import tensorflow as tf

import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

## <font color="#CA3532">Carga de los datos del problema Iris</font>

Vamos a empezar con un problema muy sencillo, uno de los problemas de clasificación más famosos que existen. Fue propuesto por R.A. Fisher en 1936, y consiste en clasificar plantas de la especie *Iris* en tres subespecies: *iris-virginica*, *iris-setosa* e *iris-versicolor*. Los atributos que describen cada planta son las dimensiones (longitud y anchura) del pétalo y el sépalo. El conjunto de datos contiene un total de 150 plantas, 50 de cada una de las clases. 

![alt text](https://drive.google.com/uc?id=1aN5FeKpb1SBUhuglGNIe97-HFB0Naiaz)

En la siguiente celda cargamos los datos del problema, que están incluidos en el paquete *sklearn.datasets*. En este enlace tienes una descripción de los datos:

https://scikit-learn.org/stable/datasets/toy_dataset.html#iris-plants-dataset
 

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()

La variable ``iris`` es un diccionario con los siguientes elementos:

- ``data``: array de numpy con los datos del problema (no incluye la clase). Cada fila es un ejemplo (150), cada columna es un atributo (4).
- ``target``: array de numpy con las clases de los 150 ejemplos, cada clase es un número entre 0 y 2.
- ``target_names``: array de numpy con los nombres de las 3 clases.
- ``DESCR``: string con una descripción del problema.
- ``feature_names``: lista con los nombres de los 4 atributos.
- ``filename``: nombre del fichero que contiene los datos.

### <font color="#CA3532">Visualización de los datos</font>

Creamos un DataFrame con los datos para que sea más fácil visualizarlos:

In [ ]:
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target_names[iris.target]
df['target_num'] = iris.target
df[::10].head(15)

Y mostramos la distribución de las clases en el plano definido por cada pareja de atributos. Como vemos el problema no es muy complicado. Una de las clases (setosa) está completamente separada de las otras dos. Las dos últimas solapan ligeramente.

In [ ]:
plt.figure(figsize=(12,15))
n_classes = 3
plot_colors = "bry"

for pairidx, pair in enumerate([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3]]):
    X = iris.data[:, pair]
    y = iris.target

    plt.subplot(3, 2, pairidx + 1)
    plt.xlabel(iris.feature_names[pair[0]])
    plt.ylabel(iris.feature_names[pair[1]])
    plt.grid(True)
        
    plt.plot(X[y==0,0], X[y==0,1], 'o', label=iris.target_names[0], color='#993300')
    plt.plot(X[y==1,0], X[y==1,1], 'o', label=iris.target_names[1], color='#009933')
    plt.plot(X[y==2,0], X[y==2,1], 'o', label=iris.target_names[2], color='#000099')

plt.legend(loc=2)
plt.show()

### <font color="#CA3532">Preparación de los datos</font>

Vamos a considerar sólo las dos últimas dimensiones del problema (la longitud y la anchura del pétalo). Esto nos permitirá visualizar el modelo en 2D. 

A continuación generamos los arrays de datos ``x`` y ``t``:

In [ ]:
x = iris.data[:, -2:]
t = iris.target
[n, d] = x.shape

Los datos del problema están ordenados (los primeros 50 ejemplos son de la clase setosa, los 50 siguientes de la clase versicolor, y los 50 últimos de la clase virginica). Para evitar sesgos durante el entrenamiento del modelo conviene desordenarlos:

In [ ]:
p = np.random.permutation(n)
x = x[p, :]
t = t[p]

Finalmente trasponemos la variable ``x`` para que tenga dimensiones ``2x150`` e introducimos una dimensión *dummy* en la variable ``t`` para que tenga dimensiones ``1x150``.

In [ ]:
x = x.T
t = t[None, :]
print("Dimension de x:", x.shape)
print("Dimension de t:", t.shape)

La figura siguiente muestra los datos del problema:

In [ ]:
plt.figure(figsize=(5, 5))

plt.plot(x[0, t.ravel()==0], x[1, t.ravel()==0], 'o', label=iris.target_names[0], color='#993300')
plt.plot(x[0, t.ravel()==1], x[1, t.ravel()==1], 'o', label=iris.target_names[1], color='#009933')
plt.plot(x[0, t.ravel()==2], x[1, t.ravel()==2], 'o', label=iris.target_names[2], color='#000099')

plt.xlabel(iris.feature_names[pair[0]])
plt.ylabel(iris.feature_names[pair[1]])
plt.grid(True)
      
plt.legend(loc=2)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Particion entrenamiento-test:
x_train, x_test, t_train, t_test = train_test_split(x.T, t.T, test_size=0.33, random_state=12)

# Estandarizacion:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

## <font color="#CA3532">Estrategia One-vs-rest</font>

Para aquellos modelos más sencillos, como la regresión logística que hemos visto, una alternativa para construir un modelo de decisión multiclase es utilizar la estrategia **One-vs-rest**.

La estrategia consiste en construir N modelos para que cada uno aprenda a identificar cada una de las N clases por separado, diferenciándolas de los demás. Una vez los modelos se han entrenado **con los mismos hiperparámetros**, se normalizan las predicciones para dar la clase con mayor probabilidad.

### <font color="#CA3532">Definición del modelo</font>

In [ ]:
class LogisticRegressionModel:

  def __init__(self, d0=2):
    self.W = tf.Variable(tf.random.normal(shape=[d0, 1], dtype=tf.dtypes.float64))  
    self.b = tf.Variable(tf.random.normal(shape=[1], dtype=tf.dtypes.float64)) 

  def predict(self, x):
    """
    x must be a (n,d0) array
    returns a (n,1) array with the predictions for each of the n patterns
    """
    z = tf.matmul(x, self.W) + self.b
    y = tf.math.sigmoid(z)
    return y

  def loss(self, x, t):
    """
    computes the cross-entropy between the model predictions and the targets
    """
    y = self.predict(x)
    loss = tf.reduce_mean(-t*tf.math.log(y) - (1.-t)*tf.math.log(1.-y), axis=0)
    return loss

  def fit(self, x, t, eta, num_epochs):
    """
    Fits the model parameters with data (x, t) using a learning rate eta and
    num_epochs epochs
    """
    loss_history = []
    for epoch in range(num_epochs):
      with tf.GradientTape() as tape:
        loss = self.loss(x, t)

      loss_history.append(loss.numpy().ravel()[0])
      
      [db, dW] = tape.gradient(loss, [self.b, self.W])
      self.b.assign(self.b - eta*db)
      self.W.assign(self.W - eta*dW)
      
    return loss_history

  def accuracy(self, x, t):
    y = self.predict(x).numpy()
    pred = y > 0.5
    return np.mean(pred == t)

Una vez definida la clase *LogisticRegressionModel*, voy a definir una clase *MulticlassLogisticRegressionModel*. Esta clase tiene que hacer lo siguiente:

- Tener un conjunto de modelos de tipo *LogisticRegressionModel*, tantos como clases diferentes a clasificar haya en el problema (en el ejemplo que estamos viendo, son 3 clases: setosa, versicolor y virginica).

- Entrenar el modelo multiclase significa entrenar cada modelo con cada clase a pares, es decir, el modelo 0 debe entrenarse para diferenciar la clase 0 de las demás; el modelo 1 debe entrenarse para diferenciar la clase 1 de las demás; y el modelo 2 debe entrenarse para diferenciar la clase 2 de las demás.

- Predecir el resultado final debe dar la probabilidad que la combinación de los tres modelos da a cada clase. Para ello, lo que vamos a hacer es normalizar las probabilidades siguiendo la siguiente ecuación:

$$P(clase=i)(x) = \frac{P_{modelo=i}(clase=i)(x)}{\sum_{j=0}^{j=N\_clases}{P_{modelo=j}(clase=j)(x)}}$$

Donde $P_{modelo=i}(clase=i)(x)$ es la probabilidad que el modelo asociado a la clase $i$ da al dato $x$ de pertenecer a la clase $i$ frente a las demás.

In [ ]:
class MulticlassLogisticRegressionModel:

  def __init__(self, input_dimension, num_classes):
    self.models = [LogisticRegressionModel(input_dimension) for _ in range(num_classes)]
    self.num_classes = num_classes

  def fit(self, x, t, eta, num_epochs):
    for id_class, model in enumerate(self.models):
      t_class = (t == id_class) * 1
      model.fit(x, t_class, eta, num_epochs)

  def predict(self, x):
    preds = np.zeros((x.shape[0], self.num_classes))
    for id_class, model in enumerate(self.models):
      preds[:, id_class] = model.predict(x)[:, 0]
    preds /= preds.sum(axis=1, keepdims=True)
    return preds

  def accuracy(self, x, t):
    preds = self.predict(x)
    y = np.argmax(preds, axis=1)[:, None]
    print(y.shape)
    return np.mean(y == t)

### <font color="#CA3532">Entrenamiento</font>

In [ ]:
# Construccion del modelo
n, d = x_train.shape
num_clases = len(np.unique(t_train))

model = MulticlassLogisticRegressionModel(d, num_clases)

# Entrenamiento del modelo
eta = 0.8
epochs = 200

model.fit(x_train, t_train, eta, epochs)

In [ ]:
preds_train = model.predict(x_train)
preds_test = model.predict(x_test)

In [ ]:
print("Accuracy (train):", model.accuracy(x_train, t_train))
print("Accuracy (test):", model.accuracy(x_test, t_test))

(100, 1)
Accuracy (train): 0.96
(50, 1)
Accuracy (test): 0.96


### <font color="#CA3532">Visualización del modelo</font>

Pintamos la decisión global del modelo One-vs-rest:

In [ ]:
xx, yy = np.meshgrid(np.arange(x_train[:, 0].min()-0.2, x_train[:, 0].max()+0.2, 0.1), 
                     np.arange(x_train[:, 1].min()-0.2, x_train[:, 1].max()+0.2, 0.1))
xy = np.concatenate([xx.reshape([1, -1]), yy.reshape([1, -1])],axis=0).T
z = model.predict(xy)
print(z.shape)

plt.figure(figsize=(16, 4))

for clase_objetivo in [0, 1, 2]:
  plt.subplot(1, 3, clase_objetivo+1)
  plt.contourf(xx, yy, z[:, clase_objetivo].reshape(xx.shape), 100, cmap="bwr", alpha=0.4, vmin=0.0, vmax=1.0)

  plt.plot(x_test[t_test.ravel()==0, 0], x_test[t_test.ravel()==0, 1], 'o', label=iris.target_names[0] if clase_objetivo == 0 else 'others', color='red' if clase_objetivo == 0 else 'blue')
  plt.plot(x_test[t_test.ravel()==1, 0], x_test[t_test.ravel()==1, 1], 'o', label=iris.target_names[1] if clase_objetivo == 1 else 'others', color='red' if clase_objetivo == 1 else 'blue')
  plt.plot(x_test[t_test.ravel()==2, 0], x_test[t_test.ravel()==2, 1], 'o', label=iris.target_names[2] if clase_objetivo == 2 else 'others', color='red' if clase_objetivo == 2 else 'blue')

  plt.title("Probabilidad de %s" % (iris.target_names[clase_objetivo]))
  plt.xlabel(iris.feature_names[pair[0]])
  plt.ylabel(iris.feature_names[pair[1]])
  plt.grid(True)
        
  plt.legend(loc=2)
  plt.colorbar()

plt.show()

Ahora pintamos la frontera de decisión de cada uno de los modelos que componen el modelo global One-vs-rest:

In [ ]:
plt.figure(figsize=(16, 4))
for clase_objetivo, m in enumerate(model.models):
  xx, yy = np.meshgrid(np.arange(x_train[:, 0].min()-0.2, x_train[:, 0].max()+0.2, 0.1), 
                     np.arange(x_train[:, 1].min()-0.2, x_train[:, 1].max()+0.2, 0.1))
  xy = np.concatenate([xx.reshape([1, -1]), yy.reshape([1, -1])],axis=0).T
  z = m.predict(xy).numpy()

  plt.subplot(1, 3, clase_objetivo+1)
  plt.contourf(xx, yy, z.reshape(xx.shape), 100, cmap="bwr", alpha=0.4, vmin=0.0, vmax=1.0)

  plt.plot(x_test[t_test.ravel()==0, 0], x_test[t_test.ravel()==0, 1], 'o', label=iris.target_names[0] if clase_objetivo == 0 else 'others', color='red' if clase_objetivo == 0 else 'blue')
  plt.plot(x_test[t_test.ravel()==1, 0], x_test[t_test.ravel()==1, 1], 'o', label=iris.target_names[1] if clase_objetivo == 1 else 'others', color='red' if clase_objetivo == 1 else 'blue')
  plt.plot(x_test[t_test.ravel()==2, 0], x_test[t_test.ravel()==2, 1], 'o', label=iris.target_names[2] if clase_objetivo == 2 else 'others', color='red' if clase_objetivo == 2 else 'blue')

  plt.title("Probabilidad de %s" % (iris.target_names[clase_objetivo]))
  plt.xlabel(iris.feature_names[pair[0]])
  plt.ylabel(iris.feature_names[pair[1]])
  plt.grid(True)
        
  plt.legend(loc=2)
  plt.colorbar()

plt.show()

## <font color="#CA3532">Modelo multiclase: Red neuronal</font>

### <font color="#CA3532">Definición del modelo</font>

In [ ]:
class NeuralNetworkModel:

  def __init__(self, layers_size=[2]):
    self.W = [tf.Variable(tf.random.normal(shape=[a, b], dtype=tf.dtypes.float64)) for a, b in zip(layers_size[:-1], layers_size[1:])]
    self.b = [tf.Variable(tf.random.normal(shape=[1, b], dtype=tf.dtypes.float64)) for b in layers_size[1:]]

  def predict_pre_activation(self, x):
    """
    x must be a (n,d0) array
    returns a (n,num_clases) array with the pre-activations for each of the n patterns
    """
    y = x
    for w, b in zip(self.W[:-1], self.b[:-1]):
      z = tf.matmul(y, w) + b
      y = tf.nn.sigmoid(z)

    z = tf.matmul(y, self.W[-1]) + self.b[-1]

    return z

  def predict(self, x):
    """
    x must be a (n,d0) array
    returns a (n,num_clases) array with the predictions for each of the n patterns
    """
    y = x
    for w, b in zip(self.W[:-1], self.b[:-1]):
      z = tf.matmul(y, w) + b
      y = tf.nn.sigmoid(z)

    z = tf.matmul(y, self.W[-1]) + self.b[-1]
    y = tf.nn.softmax(z, axis=1)
 
    return y

  def loss(self, x, t):
    """
    computes the MSE between the model predictions and the targets
    """
    z = self.predict_pre_activation(x)
    xentropy = tf.compat.v1.losses.sparse_softmax_cross_entropy(t, z) # Esta función necesita que pasemos los logits
    loss = tf.reduce_mean(xentropy)
    return loss

  def fit(self, x, t, eta, num_epochs):
    """
    Fits the model parameters with data (x, t) using a learning rate eta and
    num_epochs epochs
    """
    loss_history = []
    for epoch in range(num_epochs):
      with tf.GradientTape(persistent=True) as tape:
        loss = self.loss(x, t)

      loss_history.append(loss.numpy().ravel()[0])
      
      for b, W in zip(self.b, self.W):
        [db, dW] = tape.gradient(loss, [b, W])
        b.assign(b - eta*db)
        W.assign(W - eta*dW)
      
    return loss_history

  def accuracy(self, x, t):
    preds = self.predict(x).numpy()
    y = np.argmax(preds, axis=1)[:, None]
    return np.mean(y == t)

### <font color="#CA3532">Entrenamiento</font>

In [ ]:
nepocas = 500
eta = 0.1

model = NeuralNetworkModel([2, 20, 3])

loss = model.fit(x_train, t_train, eta, nepocas)

### <font color="#CA3532">Coste frente a número de épocas</font>

In [ ]:
plt.plot(loss)
plt.grid(True)
plt.xlabel("época")
plt.ylabel("Cross-entropy loss")
plt.show()

In [ ]:
preds_train = model.predict(x_train).numpy()
preds_test = model.predict(x_test).numpy()

In [ ]:
print("Accuracy (train):", model.accuracy(x_train, t_train))
print("Accuracy (test):", model.accuracy(x_test, t_test))

### <font color="#CA3532">Visualización del modelo</font>

In [ ]:
xx, yy = np.meshgrid(np.arange(x_train[:, 0].min()-0.5, x_train[:, 0].max()+0.5, 0.1), 
                     np.arange(x_train[:, 1].min()-0.5, x_train[:, 1].max()+0.5, 0.1))
xy = np.concatenate([xx.reshape([1, -1]), yy.reshape([1, -1])],axis=0).T
z = model.predict(xy).numpy()
print(z.shape)

plt.figure(figsize=(16, 4))

for clase_objetivo in [0, 1, 2]:
  plt.subplot(1, 3, clase_objetivo+1)
  plt.contourf(xx, yy, z[:, clase_objetivo].reshape(xx.shape), 100, cmap="bwr", alpha=0.4, vmin=0.0, vmax=1.0)

  plt.plot(x_test[t_test.ravel()==0, 0], x_test[t_test.ravel()==0, 1], 'o', label=iris.target_names[0] if clase_objetivo == 0 else 'others', color='red' if clase_objetivo == 0 else 'blue')
  plt.plot(x_test[t_test.ravel()==1, 0], x_test[t_test.ravel()==1, 1], 'o', label=iris.target_names[1] if clase_objetivo == 1 else 'others', color='red' if clase_objetivo == 1 else 'blue')
  plt.plot(x_test[t_test.ravel()==2, 0], x_test[t_test.ravel()==2, 1], 'o', label=iris.target_names[2] if clase_objetivo == 2 else 'others', color='red' if clase_objetivo == 2 else 'blue')

  plt.title("Probabilidad de %s" % (iris.target_names[clase_objetivo]))
  plt.xlabel(iris.feature_names[pair[0]])
  plt.ylabel(iris.feature_names[pair[1]])
  plt.grid(True)
        
  plt.legend(loc=2)
  plt.colorbar()

plt.show()

## <font color="#CA3532">Ejercicios adicionales</font>

- Calcular matriz de confusión
- Análisis ROC
- Precision-recall


In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve

In [ ]:
matrix = confusion_matrix(t_test[:, 0], np.argmax(preds_test, axis=1))

sn.heatmap(matrix, annot=True, xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.show()

In [ ]:
plt.figure(figsize=(16, 4))
for i in range(3):
  fpr, tpr, thresholds = roc_curve(t_test[:, 0] == i, np.argmax(preds_test, axis=1) == i)
  plt.subplot(1,3,i+1)
  plt.plot(fpr, tpr)
  plt.xlabel("False Positive Rate")
  plt.ylabel("True Positive Rate")
  plt.title("ROC Curve - Class " + iris.target_names[i])
  plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(16, 4))
for i in range(3):
  prec, recall, _ = precision_recall_curve(t_test[:, 0] == i, np.argmax(preds_test, axis=1) == i)
  print(recall, prec)
  plt.subplot(1,3,i+1)
  plt.plot(recall, prec)
  plt.xlabel("Recall")
  plt.ylabel("Precision")
  plt.title("Precision-Recall Curve - Class " + iris.target_names[i])
  plt.ylim(-0.05, 1.05)
  plt.grid()
plt.show()